In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
FAST_EMA = 15
SLOW_EMA = 22
SIGNAL_EMA = 9
STOCH_RSI_PERIOD = 7
SMOOTH_K = 3
SMOOTH_D = 2
RSI_PERIOD = 21
ATR_PERIOD = 8
ATR_MULTIPLIER = 1.5
STOP_LOSS_PERCENTAGE = 0.03
TP_LONG_PERCENTAGE = 0.075
TP_SHORT_PERCENTAGE = 0.075
WEIGHT_SIGNAL = 2
CANDLE_DELAY = 1

def macd(df):
    df['ema_fast'] = df['hl2'].ewm(span=FAST_EMA, min_periods=1).mean()
    df['ema_slow'] = df['hl2'].ewm(span=SLOW_EMA, min_periods=1).mean()
    df['macd'] = df['ema_fast'] - df['ema_slow']
    df['macd_signal'] = df['macd'].ewm(span=SIGNAL_EMA, min_periods=1).mean()

def stoch_rsi(df):
    delta = df['close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=STOCH_RSI_PERIOD).mean()
    avg_loss = loss.rolling(window=STOCH_RSI_PERIOD).mean()
    rs = avg_gain / avg_loss
    df['rsi'] = 100 - (100 / (1 + rs))
    df['stoch_rsi'] = (df['rsi'] - df['rsi'].rolling(window=STOCH_RSI_PERIOD).min()) / \
                      (df['rsi'].rolling(window=STOCH_RSI_PERIOD).max() - df['rsi'].rolling(window=STOCH_RSI_PERIOD).min())
    df['stoch_rsi_k'] = df['stoch_rsi'].rolling(window=SMOOTH_K).mean() * 100
    df['stoch_rsi_d'] = df['stoch_rsi_k'].rolling(window=SMOOTH_D).mean()

def rsi(df):
    delta = df['close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=RSI_PERIOD).mean()
    avg_loss = loss.rolling(window=RSI_PERIOD).mean()
    rs = avg_gain / avg_loss
    df['rsi'] = 100 - (100 / (1 + rs))

def supertrend(df):
    df['atr'] = df['hl2'].rolling(window=ATR_PERIOD).apply(lambda x: np.max(x) - np.min(x), raw=True)
    df['upper_band'] = (df['hl2'] + ATR_MULTIPLIER * df['atr'])
    df['lower_band'] = (df['hl2'] - ATR_MULTIPLIER * df['atr'])
    df['supertrend'] = np.where(df['close'] > df['upper_band'], df['lower_band'], df['upper_band'])

def apply_strategy(df):
    trade_log = []

    df['hl2'] = (df['high'] + df['low']) / 2
    macd(df)
    stoch_rsi(df)
    rsi(df)
    supertrend(df)
    df['ema50'] = df['close'].ewm(span=50, min_periods=1).mean()
    df['ema88'] = df['close'].ewm(span=88, min_periods=1).mean()

    in_position = False
    position_type = None
    entry_price = 0
    stop_loss = 0
    take_profit = 0
    entry_time = None

    for i in range(CANDLE_DELAY, len(df)):
        weight_strategy = 0
        if df['macd'].iloc[i] > df['macd_signal'].iloc[i]:
            weight_strategy += 1
        if df['stoch_rsi_k'].iloc[i] > 70:
            weight_strategy += 1
        if df['rsi'].iloc[i] > 70:
            weight_strategy += 1
        if df['close'].iloc[i] > df['supertrend'].iloc[i]:
            weight_strategy += 1
        if df['ema50'].iloc[i] > df['ema88'].iloc[i]:
            weight_strategy += 1

        if weight_strategy > WEIGHT_SIGNAL and not in_position:
            in_position = True
            position_type = 'long'
            entry_price = df['close'].iloc[i]
            stop_loss = df['hl2'].iloc[i] * (1 - STOP_LOSS_PERCENTAGE)
            take_profit = entry_price * (1 + TP_LONG_PERCENTAGE)
            entry_time = df['datetime'].iloc[i]

        elif weight_strategy < -WEIGHT_SIGNAL and not in_position:
            in_position = True
            position_type = 'short'
            entry_price = df['close'].iloc[i]
            stop_loss = df['hl2'].iloc[i] * (1 + STOP_LOSS_PERCENTAGE)
            take_profit = entry_price * (1 - TP_SHORT_PERCENTAGE)
            entry_time = df['datetime'].iloc[i]

        elif in_position and position_type == 'long':
            if df['close'].iloc[i] >= take_profit or df['close'].iloc[i] <= stop_loss:
                exit_price = df['close'].iloc[i]
                exit_time = df['datetime'].iloc[i]
                pnl = (exit_price - entry_price) / entry_price * 100
                trade_log.append([entry_time, entry_price, exit_time, exit_price, pnl, 'long'])
                in_position = False

        elif in_position and position_type == 'short':
            if df['close'].iloc[i] <= take_profit or df['close'].iloc[i] >= stop_loss:
                exit_price = df['close'].iloc[i]
                exit_time = df['datetime'].iloc[i]
                pnl = (entry_price - exit_price) / entry_price * 100
                trade_log.append([entry_time, entry_price, exit_time, exit_price, pnl, 'short'])
                in_position = False

    trade_log_df = pd.DataFrame(trade_log, columns=['entry_datetime', 'entry_price', 'exit_datetime', 'exit_price', 'pnl%', 'trade'])
    return trade_log_df

# Main execution
file_path = 'btc_18_22_1d.csv'  
df = pd.read_csv(file_path)

trade_log_df = apply_strategy(df)

In [3]:
df['signal'] = 0

# Loop through each row in the trade_log_df to update the original DataFrame
for index, row in trade_log_df.iterrows():
    # Set signal to 1 for entry date
    entry_date = row['entry_datetime']
    df.loc[df['datetime'] == entry_date, 'signal'] = 1
    
    # Set signal to -1 for exit date
    exit_date = row['exit_datetime']
    df.loc[df['datetime'] == exit_date, 'signal'] = -1

In [4]:
df = df[['datetime', 'close', 'signal']]

In [5]:
df['daily_returns'] = df['close'].pct_change()
df['buy_and_hold_returns'] = (1 + df['daily_returns']).cumprod()

df.loc[0, 'buy_and_hold_returns'] = 1

In [6]:
df['strategy_returns'] = 1.0
in_trade = False

for i in range(1, len(df)):
    if df['signal'].iloc[i-1] == 1:
        in_trade = True
    elif df['signal'].iloc[i-1] == -1:
        in_trade = False
    
    if in_trade:
        df.loc[i, 'strategy_returns'] = df['strategy_returns'].iloc[i-1] * (1 + df['daily_returns'].iloc[i])
    else:
        df.loc[i, 'strategy_returns'] = df['strategy_returns'].iloc[i-1]

df.loc[0, 'strategy_returns'] = 1

In [7]:
df['daily_strategy_returns'] = df['strategy_returns'].pct_change()

In [12]:
non_zero_returns = df[df['daily_strategy_returns'] != 0]['daily_strategy_returns']
sharpe_ratio = (non_zero_returns.mean() / non_zero_returns.std()) * (365 ** 0.5)

negative_returns = df[df['daily_strategy_returns'] < 0]['daily_strategy_returns']
sortino_ratio = (non_zero_returns.mean() / negative_returns.std()) * (365 ** 0.5)

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.date

plt.figure(figsize=(14, 8))
plt.plot(df['date'], df['buy_and_hold_returns'], label='Buy and Hold Returns', color='blue')
plt.plot(df['date'], df['strategy_returns'], label='Strategy Returns', color='green')

plt.xlabel('Date (by Month)')
plt.ylabel('Returns')
plt.title('Buy and Hold vs Strategy Returns')
plt.legend()

plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [11]:
trade_log_df.to_csv('trade_logs.csv')

In [13]:
print(sharpe_ratio)

4.056361407423136


In [14]:
print(sortino_ratio)

7.581961902988647
